# 最近のコンテストで動くようにする

In [47]:
import re
import requests
from bs4 import BeautifulSoup

def get_problem_data(url):
    
    print(url)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")
    problem = soup.select('#task-statement')
    hrs = problem[0].find_all("h3")
    secs = problem[0].find_all("section")

    data = {
        "URL": resp.url,
        "title": soup.title.text,
        "contest_name": soup.select(".contest-title")[0].text,
        "contest_id": resp.url.split("/")[4],
        "problem_str": soup.title.text.split(" - ")[0],
        "problem_name": soup.title.text.split(" - ")[1],
        "problem": secs[0],
        "input": secs[1],
        "output": secs[2],
        "ios": [[hr.text, sec] for hr, sec in zip(hrs[3:], secs[3:])]
    }
    return data

def replace_formula(data):
    for x in data.find_all(re.compile("var|code")):
        x.insert_before("[$ " + x.get_text())
        x.insert_after(" ]")
        x.decompose()

def replace_newline(data):
    return re.sub("\n$", "", re.sub("^\n", "", re.sub("(\r\n)+", "\n ", data.text)))

def output_one_problem_for_scrapbox(url):
    data = get_problem_data(url)
    replace_formula(data["problem"])
    replace_formula(data["input"])
    replace_formula(data["output"])
    
    print(data["contest_id"].upper() + " " + data["title"])
    print("[* 問題文]")
    print(replace_newline(data["problem"]).strip())
    print("[* 入力]")
    print(replace_newline(data["input"]).strip())
    print("[* 出力]")
    print(replace_newline(data["output"]).strip())
    for item in data["ios"]:
        print("[* " + item[0] + "]")
        replace_formula(item[1])
        # print(item[1])
        print("code:memo\n" + replace_newline(item[1]))

    print("[" + data["URL"] + " " +  data["title"] + " - " + data["contest_name"] +"]")
    print("\n\n\n")
    print(" ".join(["#AtCoder", "#" + data["contest_id"]]))

output_one_problem_for_scrapbox("https://atcoder.jp/contests/abc227/tasks/abc227_a")

https://atcoder.jp/contests/abc227/tasks/abc227_a
ABC227 A - Last Card
[* 問題文]
問題文[$ 1,2,\ldots,N ] の番号のついた [$ N ] 人の人に合計 [$ K ] 枚のカードを配ります。
人 [$ A ] から始めて 人 [$ A,A+1,A+2,\ldots,N,1,2,\ldots ] の順に [$ 1 ] 枚ずつカードを配るとき、最後のカードは誰に配られるでしょうか？
厳密には、人 [$ x(1 \leq x < N) ] の次には人 [$ x+1 ] にカードを配り、人 [$ N ] の次には人 [$ 1 ] にカードを配ります。
[* 入力]
制約
[$ 1 \leq N,K \leq 1000 ]
[$ 1 \leq A \leq N ]
入力は全て整数
[* 出力]
入力入力は以下の形式で標準入力から与えられる。
[$ N ] [$ K ] [$ A ]
[* 出力]
code:memo
出力最後のカードが配られた人の番号を出力せよ。
[* 入力例 1]
code:memo
入力例 13 3 2
 
[* 出力例 1]
code:memo
出力例 11
 
人 [$ 2 ]、人 [$ 3 ]、人 [$ 1 ] の順にカードを配ります。
[* 入力例 2]
code:memo
入力例 21 100 1
 
[* 出力例 2]
code:memo
出力例 21
 
[* 入力例 3]
code:memo
入力例 33 14 2
 
[* 出力例 3]
code:memo
出力例 33
 
[* Problem Statement]
code:memo
Problem StatementWe will hand out a total of [$ K ] cards to [$ N ] people numbered [$ 1, 2, \ldots, N ].
Beginning with Person [$ A ], we will give the cards one by one to the people in this order: [$ A, A+1, A+2, \ldots, N, 1, 2, \ldots ]. Who will get the la

# atcoder-toolsを利用

In [32]:
# del atcodertools.client.atcoder
del AtCoderClient

In [7]:
def get_methods(object, spacing=20):
  methodList = []
  for method_name in dir(object):
    try:
        if callable(getattr(object, method_name)):
            methodList.append(str(method_name))
    except:
        methodList.append(str(method_name))
  processFunc = (lambda s: ' '.join(s.split())) or (lambda s: s)
  for method in methodList:
    try:
        print(str(method.ljust(spacing)) + ' ' +
              processFunc(str(getattr(object, method).__doc__)[0:90]))
    except:
        print(method.ljust(spacing) + ' ' + ' getattr() failed')

In [9]:
# from importlib import reload
from atcodertools.client.models.contest import Contest
from atcodertools.client.models.problem import Problem
from atcodertools.client.models.problem_content import ProblemContent, remove_non_jp_characters
from atcodertools.client.atcoder import AtCoderClient
import requests
from bs4 import BeautifulSoup


# import inspect
# stack = inspect.stack()
# for s in stack[1:]:
#     m = inspect.getmodule(s[0])
#     if m:
#         print(m.__file__)
#         # break
# print(inspect.getframeinfo(inspect.getouterframes(inspect.currentframe())[1][0])[0])

# get_methods(AtCoderClient)
def scrape_all_tasks(contest_id):
    client = AtCoderClient()
    # client.clear()
    contest = client.scrape_contest_info(contest_id=contest_id)
    # サイトデータ取得で構築できる分を構築
    # 全問題を取得
    problem_list = client.download_problem_list(contest)
    
    # タスクの取得
    tasks = [(client,
          problem) for
          problem in problem_list]

    contest = Contest(contest_id)
    # 不足分の取得
    resp = client._request(tasks[0][1].get_url())
    soup = BeautifulSoup(resp.text, "html.parser")
    for e in soup.findAll("span", {"class": "lang-en"}):
        e.extract()
    problem = soup.select('#task-statement')
    # print(problem)
    # print("============================")
    # tmp = soup.select('.part')
    # print(tmp)
    # hrs = problem[0].find_all("h3")
    # secs = problem[0].find_all("section")
    task_statement = []
    restriction_tags = []
    input_tags = []
    input_format_tag = None
    output_tags = []
    for tag in soup.select('section'):
        h3tag = tag.find('h3')
        if h3tag is None:
            continue
        # Some problems have strange characters in h3 tags which should be
        # removed
        section_title = remove_non_jp_characters(tag.find('h3').get_text())
        print(section_title)
        if section_title.startswith("問題文"):
            task_statement = tag.select('h3 ~ p')
        elif section_title.startswith("制約"):
            restriction_tags = tag.find("ul").select("li")
        elif section_title.startswith("入力例"):
            input_tags.append(tag.find('pre'))
        elif section_title.startswith("入力"):
            input_format_tag = tag.find('pre')
        elif section_title.startswith("出力例"):
            output_tags.append(tag.find('pre'))
    print("1")
    print(task_statement)
    print("2")
    print(restriction_tags)
    print("3")
    print(input_tags)
    print("4")
    print(output_tags)
    # print(data["contest_id"].upper() + " " + data["title"])
    # print("[* 問題文]")
    # print(replace_newline(data["problem"]).strip())
    # print("[* 入力]")
    # print(replace_newline(data["input"]).strip())
    # print("[* 出力]")
    # print(replace_newline(data["output"]).strip())
    

scrape_all_tasks("abc220")

問題文
制約
入力
出力
入力例
出力例
入力例
出力例
1
[<p><var>A</var> 以上 <var>B</var> 以下であるような <var>C</var> の倍数を、<var>1</var> つ出力してください。  </p>, <p>条件を満たす数が存在しない場合は <code>-1</code> を出力してください。</p>]
2
[<li><var>1 \leq A \leq B \leq 1000</var></li>, <li><var>1 \leq C \leq 1000</var></li>, <li>入力は全て整数</li>]
3
[<pre>123 456 100
</pre>, <pre>630 940 314
</pre>]
4
[<pre>200
</pre>, <pre>-1
</pre>]
